# Dendograms between heatmaps

Compute dendograms to show similarities between K-Cross Validation trainings, networks and expressions. 
- cv2.TM_CCOEFF_NORMED (normalized correlation coefficient) is used to compute the distance between two masks.

In [11]:
import os
import cv2

from ekman_expressions.heatmaps import plot_dendogram

In [22]:
# Models to evaluate
model_names = ['SilNet', 'WeiNet', 'AlexNet', 'SongNet', 'InceptionV3',
               'VGG19', 'VGG16', 'ResNet50', 'ResNet101V2', 'Xception',
               'MobileNetV3Large', 'EfficientNetV2B0']
model_names2 = ['SilNet', 'WeiNet', 'AlexNet', 'SongNet', 'InceptionV3',
               'VGG19', 'VGG16', 'ResNet50', 'ResNet101V2', 'Xception',
               'MobileNetV3', 'EfficientNetV2']

# Number of k-cross validations and folder where they are located
# Alternatively set the paths to the target training and test manually
K = 5
dataset_paths_root = '../datasets/'
dataset_paths_train = []
dataset_paths_test = []
for i in range(K):
    dataset_paths_train.append(dataset_paths_root + 'CV' + str(i+1))
    dataset_paths_test.append(dataset_paths_root + 'CV' + str(i+1) + '_test')

# Folders where the heatmaps are saved
heatmaps_dir_gray = '../heatmaps/GRAY'

# Folders where to store dendograms and barplots
dendograms_dir = '../dendograms'

# Labels of the classes
label_names = ['Anger', 'Disgust', 'Fear', 'Happiness', 'Sadness', 'Surprise']

## Load heatmaps and create lists

In [23]:
# Subdirectories of heatmaps
heatmaps_cv_path = os.path.join(heatmaps_dir_gray, 'heatmaps_by_cv')
heatmaps_model_path = os.path.join(heatmaps_dir_gray, 'heatmaps_by_net')
heatmaps_total_path = os.path.join(heatmaps_dir_gray, 'heatmaps_total')

# Name and distance for each heatmap
cv_names_list = [[] for _ in range(len(label_names))]
cv_heats_list = [[] for _ in range(len(label_names))]
net_names_list = [[] for _ in range(len(label_names))]
net_heats_list = [[] for _ in range(len(label_names))]
exp_net_names_list = []
exp_net_heats_list = []
exp_names_list = []
exp_heats_list = []
    
for class_i, expression in enumerate(label_names):
    
    for model_name, model_name2 in zip(model_names, model_names2):
        
        for train_path in dataset_paths_train:
            
            # By CV subsets
            # Load heatmap and compute distance
            h_name = model_name + '_' + os.path.basename(train_path) + '_' + expression + '_heatmap.png'
            heatmap = cv2.imread(os.path.join(heatmaps_cv_path, h_name), 0)
            
            # Store name and distance
            cv_names_list[class_i].append(model_name + '_' + os.path.basename(train_path))
            cv_heats_list[class_i].append(heatmap)
        
        # By net, and by net and expression
        # Load heatmap and compute distance
        h_name = model_name + '_' + expression + '_heatmap.png'
        heatmap = cv2.imread(os.path.join(heatmaps_model_path, h_name), 0)
        if heatmap is None:
            print(os.path.join(heatmaps_model_path, h_name))

        # Store name and distance
        net_names_list[class_i].append(model_name2)
        net_heats_list[class_i].append(heatmap)

        # Store name and distance
        exp_net_names_list.append(expression + '_' + model_name)
        exp_net_heats_list.append(heatmap)

    # Total
    # Load heatmap and compute distance
    h_name = expression + '_heatmap.png'
    heatmap = cv2.imread(os.path.join(heatmaps_total_path, h_name), 0)

    # Store name and distance
    exp_names_list.append(expression)
    exp_heats_list.append(heatmap)

## Compute and plot dendograms

In [ ]:
for class_i, expression in enumerate(label_names):
    
    # By net
    plot_dendogram(
        labels=net_names_list[class_i],
        heatmaps=net_heats_list[class_i],
        title=expression,
        save_path=os.path.join(dendograms_dir, expression + '_by_net'),
        distance_method=cv2.TM_CCOEFF_NORMED,
        linkage_method='average', 
        width=8, 
        height=10, 
        title_size=36, 
        font_size=24,
        lims=[0,1],
        leaf_rotation=45
    )
    
    # By CV set
    plot_dendogram(
        labels=cv_names_list[class_i],
        heatmaps=cv_heats_list[class_i],
        title=expression + ' by CV set',
        save_path=os.path.join(dendograms_dir, expression + '_by_CV'),
        distance_method=cv2.TM_CCOEFF_NORMED,
        linkage_method='average', 
        width=30, 
        height=35, 
        title_size=40, 
        font_size=16
    )

# By expression
plot_dendogram(
    labels=exp_names_list,
    heatmaps=exp_heats_list,
    title='Distance between expressions',
    save_path=os.path.join(dendograms_dir, 'total_expression'),
    distance_method=cv2.TM_CCOEFF_NORMED, 
    linkage_method='average', 
    width=8, 
    height=5, 
    title_size=16, 
    font_size=12
)

# By expression and net
plot_dendogram(
    labels=exp_net_names_list,
    heatmaps=exp_net_heats_list,
    title='Distance between expressions from different nets',
    save_path=os.path.join(dendograms_dir, 'total_net_expression'),
    distance_method=cv2.TM_CCOEFF_NORMED, 
    linkage_method='average', 
    width=30, 
    height=35, 
    title_size=40, 
    font_size=16
)